In [ ]:
import pandas as pd


def _make_unique_index(df: pd.DataFrame, key: str) -> pd.DataFrame:
    """Use `key` as index; if duplicates exist, add a tie-breaker so the index is unique."""
    if df[key].duplicated().any():
        return df.assign(_row=df.groupby(key).cumcount()).set_index([key, "_row"])
    return df.set_index(key)


def diff_by_key(df1: pd.DataFrame, df2: pd.DataFrame, key: str) -> pd.DataFrame:
    """
    Returns side-by-side differences (self/other) between df1 and df2,
    aligning rows by `key`. Handles duplicate keys and non-overlapping columns.
    """
    a = _make_unique_index(df1, key)
    b = _make_unique_index(df2, key)

    # Keep only columns present in both
    common_cols = a.columns.intersection(b.columns)
    a = a.loc[:, common_cols]
    b = b.loc[:, common_cols]

    # Make indexes identical (union, then reindex both), and sort for stability
    union_idx = a.index.union(b.index)
    a = a.reindex(union_idx).sort_index()
    b = b.reindex(union_idx).sort_index()

    # Now they are identically labeled -> safe to compare
    return a.compare(b, align_axis=0)

In [ ]:
shopify = pd.read_csv("records/shopify.csv")
shopify = shopify.rename(
    columns={
        "Order name": "name",
        "Gross sales": "grossSale",
        "Net sales": "netSale",
        "Taxes": "tax",
        "Discounts": "discount",
    }
)[["name", "grossSale", "netSale", "tax", "discount"]]

shopify = shopify.set_index("name")

display(shopify)

In [ ]:
erp = pd.read_csv("records/erp.csv")
erp = erp.groupby("name")[["grossSale", "netSale", "tax", "discount"]].sum()

display(erp)

In [113]:
merged = erp.merge(shopify, on="name", suffixes=("_erp", "_shopify"))
merged["same"] = merged["netSale_shopify"] == merged["netSale_erp"]
display(merged[merged["same"] != True][["netSale_erp", "netSale_shopify"]])

,netSale_erp,netSale_shopify
name,,
#1243754,-7.987000e+01,-77.09
#1318231,-1.678000e+01,-21.02
#1406884,-6.727000e+01,-67.29
#1407580,1.156200e+02,59.95
#1407790,1.155000e+02,59.95
#1408088,1.290500e+02,64.90
#1408139,1.665200e+02,79.96
#1408207,5.995000e+01,59.95
#1408292,1.421085e-14,0.00
